In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

In [ ]:
doubleChannel = False

file_ch0 = "output-wave0.h5"
f_ch0 = h5py.File( file_ch0, 'r')
print ( f_ch0.keys() )

f_ch1 = None
if doubleChannel:
    file_ch1 = "output-wave1.h5"
    f_ch1 = h5py.File( file_ch1, 'r')
    print ( f_ch1.keys() )

In [ ]:
dset_ch0 = f_ch0['Waveform']
dset_metadata_ch0 = f_ch0['Metadata']
print( dset_metadata_ch0 )
print ( dset_ch0 )

dset_ch1 = None
dset_metadata_ch1 = None
if doubleChannel:
    dset_ch1 = f_ch1['Waveform']
    dset_metadata_ch1 = f_ch1['Metadata']
    print( dset_metadata_ch1 )
    print ( dset_ch1 )

In [ ]:
df_ch0 = pd.DataFrame( columns=('Event','Channel','Waveform') )
df_ch0['Event']   = dset_metadata_ch0[:,0]
df_ch0['Channel'] = dset_metadata_ch0[:,1]
for i in range( dset_ch0.shape[0] ):
    df_ch0[ 'Waveform' ].iloc[ i ] = dset_ch0[ i ]
df_ch0 = df_ch0.set_index( 'Event' )
df_ch0

In [ ]:
df_ch1 = None
if doubleChannel:
    df_ch1 = pd.DataFrame( columns=('Event','Channel','Waveform') )
    df_ch1['Event']   = dset_metadata_ch1[:,0]
    df_ch1['Channel'] = dset_metadata_ch1[:,1]
    for i in range( dset_ch1.shape[0] ):
        df_ch1[ 'Waveform' ].iloc[ i ] = dset_ch1[ i ]
    df_ch1 = df_ch1.set_index( 'Event' )
    df_ch1

In [ ]:
f_ch0.close()
if doubleChannel:
    f_ch1.close()

In [ ]:
df_all = None
if doubleChannel:
    df_all = df_ch0.join( df_ch1, how='inner', lsuffix='_ch0', rsuffix='_ch1' )
    df_all

In [ ]:
conv = 4.0

i_evt = 80

event_ch0 = df_all.loc[ i_evt, 'Waveform_ch0' ]
print ( event_ch0 )

baseline_ch0 = np.mean( event_ch0[400:] )
event_ch0_corr = event_ch0 - baseline_ch0
print ( event_ch0_corr )

if doubleChannel:
    event_ch1 = df_all.loc[ i_evt, 'Waveform_ch1' ]
    print ( event_ch1 )

    baseline_ch1 = np.mean( event_ch1[400:] )
    event_ch1_corr = event_ch1 - baseline_ch1
    print ( event_ch1_corr )

fig = plt.figure( figsize=(8,6) )
X = conv * np.arange( 1024 ) 
plt.plot( X, event_ch0_corr )
if doubleChannel:
    plt.plot( X, event_ch1_corr )
plt.xlim(0,1000)

In [ ]:
from scipy import interpolate

X_inter = np.linspace( X[0], X[1023], 50*len(X) )

f_ch0 = interpolate.interp1d(X, event_ch0_corr, 'cubic')
y_inter_ch0 = f_ch0( X_inter )

if doubleChannel:
    f_ch1 = interpolate.interp1d(X, event_ch1_corr, 'cubic')
    y_inter_ch1 = f_ch1( X_inter )

fig = plt.figure( figsize=(8,6) )
plt.plot( X_inter, y_inter_ch0 )
if doubleChannel:
    plt.plot( X_inter, y_inter_ch1 )
plt.xlim(0,1000)

In [ ]:
run_interpolation = True
wave_threshold = 0.30 # percentage

# # Função retorna retorna False se existe mais de um pico ou nenhum pico que ultrapassa o threshold
# def verify_data(bin):
#     i = 0
#     bin_threshold = 0 
#     for value in bin:
#         i+=1
#         if value != 0:
#             #print(i) #debug
#             if i-bin_threshold>1 & bin_threshold!=0:
#                 return False
#             bin_threshold = i
#     if bin_threshold == 0:
#         return False
#     return True

max_vals_ch0 = []
risetimes_ch0 = []

max_vals_ch1 = None
risetimes_ch1 = None
if doubleChannel:
    max_vals_ch1 = []
    risetimes_ch1 = []

X_inter = None
if run_interpolation:
    X_inter = np.linspace( X[0], X[1023], 50*len(X) )

#descarta eventos que tenham zero ou mais de um pico acima do threshold
for i_evt_ in df_all.index:
    event_ch0_ = df_all.loc[i_evt_, 'Waveform_ch0']
    event_ch1_= None
    if doubleChannel:
        event_ch1_ = df_all.loc[i_evt_, 'Waveform_ch1']

    baseline_ch0_ = np.mean( event_ch0_[400:])
    event_ch0_corr_ = ( event_ch0_  - baseline_ch0_ )

    baseline_ch1_ = None
    event_ch1_corr_ = None
    if doubleChannel:
        baseline_ch1_ = np.mean( event_ch1_[400:])
        event_ch1_corr_ = ( event_ch1_  - baseline_ch1_ )

    binX_ch0_ = None
    max_ch0_ = None
    binX_ch1_ = None
    max_ch1_ = None
    if run_interpolation:
        f_ch0_ = interpolate.interp1d(X, event_ch0_corr_, 'cubic')
        y_inter_ch0_ = f_ch0_( X_inter )
        max_ch0_ = np.max( event_ch0_corr_ )
        binX_ch0_ = ( y_inter_ch0_ > ( wave_threshold * max_ch0_ ) ).argmax()

        if doubleChannel:
            f_ch1_ = interpolate.interp1d(X, event_ch1_corr_, 'cubic')
            y_inter_ch1_ = f_ch1_( X_inter )
            max_ch1_ = np.max( event_ch1_corr_ )
            binX_ch1_ = ( y_inter_ch1_ > ( wave_threshold * max_ch1_ ) ).argmax()
    else:
        max_ch0_ = np.max( event_ch0_corr_ )
        binX_ch0_ = ( event_ch0_corr_ > ( wave_threshold * max_ch0_ ) ).argmax()
        if doubleChannel:
            max_ch1_ = np.max( event_ch1_corr_ )
            binX_ch1_ = ( event_ch1_corr_ > ( wave_threshold * max_ch1_ ) ).argmax()        
    
    if binX_ch0_ > 0:
        max_vals_ch0.append( max_ch0_ )
        if run_interpolation:
            risetimes_ch0.append( X_inter[ binX_ch0_ ] )
        else:
            risetimes_ch0.append( X[ binX_ch0_ ] )
            
    if doubleChannel:
        if binX_ch1_ > 0:
            max_vals_ch1.append( max_ch1_ )
            if run_interpolation:
                risetimes_ch1.append( X_inter[ binX_ch1_ ] )
            else:
                risetimes_ch1.append( X[ binX_ch1_ ] )

max_vals_ch0 = np.array( max_vals_ch0 )
risetimes_ch0 = np.array( risetimes_ch0 )
if doubleChannel:
    max_vals_ch1 = np.array( max_vals_ch1 )
    risetimes_ch1 = np.array( risetimes_ch1 )

print ( max_vals_ch0 )
print ( risetimes_ch0 )
print ( len(risetimes_ch0) )
if doubleChannel:
    print ( max_vals_ch1 )
    print ( risetimes_ch1 )
    print ( len(risetimes_ch1) )

In [ ]:
if doubleChannel:
    fig = plt.figure( figsize=(8,6) )
    diff_times = ( risetimes_ch0 - risetimes_ch1 )
    plt.hist( diff_times, bins=50 )